# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# File path to the stored city data
output_data_file = os.path.join("..", "WeatherPy", "output_data", "cities.csv")

# Read cities weather data into a DataFrame
df_cities = pd.read_csv(output_data_file).drop(columns="Unnamed: 0")
df_cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,New Norfolk,58.0,AU,1.587163e+09,77.0,-42.78,147.06,285.37,1.34
1,Souillac,40.0,MU,1.587163e+09,83.0,-20.52,57.52,297.59,4.10
2,Saldanha,58.0,ZA,1.587163e+09,72.0,-33.01,17.94,288.15,4.10
3,Hilo,40.0,US,1.587163e+09,54.0,19.73,-155.09,300.15,4.10
4,St. Pete Beach,1.0,US,1.587163e+09,65.0,27.73,-82.74,302.04,3.10
...,...,...,...,...,...,...,...,...,...
548,La Macarena,88.0,CO,1.587163e+09,88.0,2.18,-73.78,296.23,1.45
549,Tucuruí,100.0,BR,1.587163e+09,93.0,-3.77,-49.67,296.33,1.09
550,Sibolga,93.0,ID,1.587163e+09,79.0,1.74,98.78,298.28,1.91
551,Kloulklubed,90.0,PW,1.587163e+09,94.0,7.04,134.26,298.15,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations and Humidity as the weights
locations = df_cities[["Lat", "Lon"]]
weights = df_cities["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# As the weather data for cities was stored with temp in Kelvins
# Let's define a conversion function, and test it here
def toK(fahr):
    return (fahr - 32.0) / 1.8 + 273.15

print(toK(32), toK(-40))

273.15 233.14999999999998


In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
hotel_df = df_cities.loc[
    df_cities["Max Temp"].between(toK(68), toK(76)) &
    (df_cities["Wind Speed"] < 5) &
    (df_cities["Cloudiness"] == 0)
].copy()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
86,Wagar,0.0,SD,1.587163e+09,33.0,16.15,36.20,295.00,3.99
144,Quelimane,0.0,MZ,1.587163e+09,88.0,-17.88,36.89,297.31,1.63
161,Pathein,0.0,MM,1.587163e+09,84.0,16.78,94.73,297.14,1.69
248,Manyana,0.0,BW,1.587163e+09,23.0,-23.40,21.72,294.82,3.32
338,Rafaela,0.0,AR,1.587163e+09,34.0,-31.25,-61.49,293.71,3.13
439,Beloha,0.0,MG,1.587163e+09,68.0,-25.17,45.05,295.32,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a new column in the dataframe. Trying to avoid a SettingWithCopyWarning warning
#pd.concat([hotel_df, pd.DataFrame({"Hotel Name1":""},index=0)], axis=1)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
86,Wagar,0.0,SD,1.587163e+09,33.0,16.15,36.20,295.00,3.99,
144,Quelimane,0.0,MZ,1.587163e+09,88.0,-17.88,36.89,297.31,1.63,
161,Pathein,0.0,MM,1.587163e+09,84.0,16.78,94.73,297.14,1.69,
248,Manyana,0.0,BW,1.587163e+09,23.0,-23.40,21.72,294.82,3.32,
338,Rafaela,0.0,AR,1.587163e+09,34.0,-31.25,-61.49,293.71,3.13,
439,Beloha,0.0,MG,1.587163e+09,68.0,-25.17,45.05,295.32,1.59,


In [8]:
# find the closest hotel

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
#    "radius": 500000,
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get coordinates from df
    loc = f"{row['Lat']},{row['Lon']}"

    # add location to params dict
    params['location'] = loc

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Retrieving Results for Index 86: Wagar.
Closest hotel is مسجد انصار السنة المحمدية.
Retrieving Results for Index 144: Quelimane.
Closest hotel is BARBEARIA BM.
Retrieving Results for Index 161: Pathein.
Closest hotel is Mi Ba Myit Tar.
Retrieving Results for Index 248: Manyana.
Missing field/result... skipping.
Retrieving Results for Index 338: Rafaela.
Closest hotel is Defensoria Del Pueblo.
Retrieving Results for Index 439: Beloha.
Missing field/result... skipping.


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
86,Wagar,0.0,SD,1.587163e+09,33.0,16.15,36.20,295.00,3.99,مسجد انصار السنة المحمدية
144,Quelimane,0.0,MZ,1.587163e+09,88.0,-17.88,36.89,297.31,1.63,BARBEARIA BM
161,Pathein,0.0,MM,1.587163e+09,84.0,16.78,94.73,297.14,1.69,Mi Ba Myit Tar
248,Manyana,0.0,BW,1.587163e+09,23.0,-23.40,21.72,294.82,3.32,
338,Rafaela,0.0,AR,1.587163e+09,34.0,-31.25,-61.49,293.71,3.13,Defensoria Del Pueblo
439,Beloha,0.0,MG,1.587163e+09,68.0,-25.17,45.05,295.32,1.59,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locs = hotel_df[["Lat", "Lon"]]

In [10]:
# Add marker layer ontop of heat map
layer_markers = gmaps.marker_layer(hotel_locs, info_box_content=hotel_info)
fig.add_layer(layer_markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))